In [3]:
import boto3
import os
import numpy as np
import uuid
import cv2 as cv
from PIL import Image
from matplotlib import pyplot as plt
# from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from io import BytesIO
import zipfile

ModuleNotFoundError: No module named 'osgeo'

In [ ]:
# Set Variables
output_bucket = "csu-team04-inference-output"
input_bucket = "csu-team04-inference-input"
model_bucket_name = "csu-team04-inference-model"
img_object_name = "output_91.tiff"
model_object_name = "new_model.h5"

In [ ]:
# Overlay Mask on Image
colour = [66, 255, 73]
colour = [i/255.0 for i in colour]

def place_mask_on_image(image, mask):
    np.place(image[:, :, :], mask[:, :, :] >= 0.5, colour)
    return image

In [ ]:
def predict(prediction):
    prediction = prediction[0][:, :, :]
    prediction = np.repeat(prediction, 3, 2)
    return prediction

In [ ]:
# Get Image From Bucket
img_file_path = f"/tmp/{uuid.uuid4()}-{img_object_name}"

s3 = boto3.client("s3")
s3.download_file(input_bucket, img_object_name, img_file_path)
image = cv.imread(img_file_path)

In [ ]:
# Get Image As Numpy Array
print("Formatting image...")
img_shape = (512, 512)
image = image[:,:,:3]
image = np.expand_dims(cv.resize(image, img_shape), 0)
image = (image/255.0)
image = image.astype(np.float32)
display(Image.open(img_file_path))

In [ ]:
# Load Model
# import tarfile
print("Loading model...")
model_path = f"/tmp/{model_object_name}"
s3.download_file(model_bucket_name, model_object_name, model_path)
# model_file = tarfile.open(model_path)
# model_file.extractall("/tmp/extracted_model")
model = load_model("/tmp/new_model.h5", compile=False)
print("Model loaded successfully!")

In [ ]:
# Make Prediction
prediction = predict(model(image))
print(prediction)

In [ ]:
# Convert Results to Tiff
upload_path = f"/tmp/{img_object_name}"
result_image = Image.fromarray((prediction * 255).astype(np.uint8))
result_image.save(upload_path)
display(Image.open(upload_path))

In [ ]:
# Display Overlay
plt.subplot(1, 2, 1)
plt.title("Satellite")
plt.imshow(image[0])
plt.subplot(1, 2, 2)
plt.title("Predicted Mask")
prediction = place_mask_on_image(image[0], prediction)
plt.imshow(prediction)
plt.show()

In [ ]:
# Upload Result to Bucket
s3.upload_file(upload_path, output_bucket, f"{img_object_name.split('.')[0]}-mask.tiff")
print("Uploaded image to bucket!")